In [1]:
import numpy as np
import pandas as pd
import torch
import csv
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
torch.set_printoptions(edgeitems=5, precision=2, linewidth=75)

In [2]:
wine_path = "/home/bartek/dlwpt-code-master/data/p1ch4/tabular-wine/winequality-white.csv"
wine_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",skiprows=1)

In [3]:
col_list = next(csv.reader(open(wine_path),delimiter=";"))

wineq=torch.from_numpy(wine_numpy)
data=wineq[:,:-1]
target=wineq[:,-1]
target=target.long()

target_onehot=torch.zeros(target.shape[0],10)
target_onehot=torch.zeros(target.shape[0],10)
target=target.unsqueeze(1)
target_onehot.scatter_(1,target,1)

data_mean=torch.mean(data,dim=0)
data_var=torch.var(data,dim=0)
data_normalized=(data-data_mean)/torch.sqrt(data_var)
data_normalized.shape

x = data_normalized.clone().detach().requires_grad_(True)
x = x.to(device='cuda')

y = target_onehot.clone().detach().requires_grad_(True)
y = y.to(device = 'cuda')

In [4]:
avg_y =sum(target)/len(target)

In [5]:
dataset = TensorDataset(x, y)
train_ds, val_ds = torch.utils.data.random_split(dataset,[4000,898])
batch_size=4898
train_loader = DataLoader(train_ds,batch_size,shuffle=False)
val_loader = DataLoader(val_ds,batch_size)

In [6]:
class Net(nn.Module):
    def __init__(self):
        
        self.device = torch.device("cuda")
        super(Net, self).__init__()
        self.input_linear=nn.Linear(11,25)
        self.midle_linear1=nn.Linear(25,50)
        self.midle_linear9=nn.Linear(50,25)
        self.output_linear=nn.Linear(25,10)
    
    def forward(self, x):
        h_relu = self.input_linear(x)
        h_relu = torch.relu(self.midle_linear1(h_relu))
        h_relu = torch.relu(self.midle_linear9(h_relu))
        y_pred = self.output_linear(h_relu)

        return y_pred

In [7]:
model = Net().to(device="cuda")
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [8]:
pozytywne = 0
suma = 0
target = target.to(device='cuda')

In [9]:
def train(n_epoch,train_loader, avg_y):
    for epoch in range(n_epoch):
        for wine_x, wine_y in train_loader:
            batch_size = wine_x.shape[0]
            y_pred = model(wine_x.view(batch_size,-1))
            loss = criterion(y_pred,wine_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        y_pred_max=torch.argmax(y_pred, dim=1).unsqueeze(1).to(device='cuda')
        y_pred_onehot=torch.zeros(y_pred_max.shape[0],10).to(device='cuda')
        y_pred_onehot.scatter_(1,y_pred_max,1)
        bledy = sum(sum(torch.abs(y_pred_onehot-wine_y)))/2

        if epoch%5 == 0 or epoch == n_epoch-1:
            print("epoch = %d, loss = %d, blad = %d" % (epoch, int(loss.item()), int(bledy.item())))
        if epoch == n_epoch-1:
            print("avg loss = %f%%" % (int(loss.item())/y_pred_max.shape[0]/avg_y*100))
            third_tensor = torch.cat((y_pred_max, torch.argmax(wine_y,1).unsqueeze(1)), 1)
            third_tensor = third_tensor.to(device = 'cpu')
            third_tensor_np = third_tensor.numpy()
            third_tensor_df = pd.DataFrame(third_tensor_np)
            third_tensor_df.columns=["y_pred_max","torch.argmax(wine_y,1).unsqueeze(1)"]
            third_tensor_df.to_csv('/home/bartek/third_tensor.csv')

In [10]:
with torch.no_grad():
    def test(val_loader,avg_y):
        for wine_x, wine_y in val_loader:
            batch_size = wine_x.shape[0]
            y_pred = model(wine_x.view(batch_size,-1))
            loss = criterion(y_pred,wine_y)

        y_pred_max=torch.argmax(y_pred, dim=1).unsqueeze(1).to(device='cuda')
        y_pred_onehot=torch.zeros(y_pred_max.shape[0],10).to(device='cuda')
        y_pred_onehot.scatter_(1,y_pred_max,1)
        bledy = sum(sum(torch.abs(y_pred_onehot-wine_y)))/2

        print("loss = %d, blad = %d" % (int(loss.item()), int(bledy.item())))
        print("avg loss = %f%%" % (int(loss.item())/y_pred_max.shape[0]/avg_y*100))

In [11]:
n_epoch = 30
train(n_epoch,train_loader,avg_y)

epoch = 0, loss = 4094, blad = 3623
epoch = 5, loss = 2604, blad = 2200
epoch = 10, loss = 2525, blad = 1989
epoch = 15, loss = 2473, blad = 1908
epoch = 20, loss = 2438, blad = 1883
epoch = 25, loss = 2415, blad = 1870
epoch = 29, loss = 2400, blad = 1845
avg loss = 10.207712%


In [12]:
test(val_loader,avg_y)

loss = 542, blad = 412
avg loss = 10.268336%
